# Pour regrouper des achats par SCC (Strongly Connected Components) https://www.geeksforgeeks.org/strongly-connected-components/

Deniz Sert
12/06/23

In [9]:
from tqdm.notebook import tqdm as progressbar
import numpy as np
import pprint
from numba import jit, njit

In [2]:
import pandas as pd
import numpy as np

df_adjacency_list = pd.read_parquet('early_adj.parquet')


# Create dummy data points
# data = [
#     [1, 2, 100, 10],
#     [2, 1, 50, 15],
#     [3, 4, 200, 20],
#     [4, 5, 150, 25],
#     [5, 6, 300, 30],
#     [6, 5, 75, 35],
#     [7, 8, 250, 40],
#     [8, 9, 125, 45],
#     [9, 10, 400, 50],
#     [10, 11, 175, 55],
#     [11, 10, 225, 60],
#     [12, 13, 350, 65],
#     [13, 14, 275, 70],
#     [14, 15, 125, 75],
#     [15, 1, 275, 80],
#     [16, 17, 150, 85],
#     [17, 18, 200, 90],
#     [18, 15, 100, 95],
#     [19, 2, 400, 100],
#     [20, 1, 50, 105]
# ]

# Create a DataFrame with the dummy data
# df_dummy = pd.DataFrame(data, columns=["addr_id1", "addr_id2", "value", "height"]).astype(
#     {
#         "addr_id1": "UInt32",
#         "addr_id2": "UInt32",
#         "value": "UInt64",
#         "height": "UInt32",
#     }
# )

# Append the DataFrame to the adjacency list
# df_adjacency_list = df_dummy
# print(df_adjacency_list)

In [3]:
from collections import defaultdict

def cluster_nodes(adjacency_list):
    dag = create_directed_graph(adjacency_list)
    clusters = identify_clusters(dag)
    
    # Analyze clusters and identify specific characteristics
    
    return clusters

def create_directed_graph(adjacency_list):
    dag = defaultdict(list)
    for entry in adjacency_list:
        source = entry["addr_id1"]
        destination = entry["addr_id2"]
        value = entry["value"]
        height = entry["height"]
        dag[source].append((destination, value, height))
    
    return dag

def identify_clusters(dag):
    # Use Strongly Connected Components (SCC) algorithm
    # to identify clusters within the DAG
    clusters = []
    
    # Implement SCC algorithm here
    
    return clusters

## SCC peut été implementé par l’algorithme de Tarjan

In [13]:
def tarjans_algorithm(graph):
    index_counter = [0]
    stack = []
    lowlink = {}
    indices = {}
    visited = set()
    result = []

    def strongconnect(node):
        indices[node] = index_counter[0]
        lowlink[node] = index_counter[0]
        index_counter[0] += 1
        stack.append(node)
        visited.add(node)

        successors = list(graph.loc[graph['addr_id1'] == node]['addr_id2'])
        for successor in successors:
            if successor not in visited:
                strongconnect(successor)
                lowlink[node] = min(lowlink[node], lowlink[successor])
            elif successor in stack:
                lowlink[node] = min(lowlink[node], indices[successor])

        if lowlink[node] == indices[node]:
            scc = []
            while True:
                successor = stack.pop()
                scc.append(successor)
                if pd.isnull(successor) or successor == node:
                    break
            result.append(scc)

    for node in progressbar(graph['addr_id1'].unique()):
        if node not in visited:
            strongconnect(node)

    return result


# Example usage
strongly_connected_components = np.array(tarjans_algorithm(df_adjacency_list[:1000]))
# print all the elements in the array whose length are greater than 1
print([x for x in strongly_connected_components if len(x) > 1])

  0%|          | 0/977 [00:00<?, ?it/s]

[]
